# Training module

In [2]:
#import library
import csv
import pandas as pd
import pickle 
import numpy as np
import cv2
import mediapipe as mp
import time
import shutil
import os
import sys
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score 

In [1]:
prj_path = r"C:\Users\Chale\Desktop\Project ITM65\Model Framework\Code\Web-applications-for-elderly-physical-exercises\ElderFitmodel"

In [3]:
# Grabbing the Holistic Model from Mediapipe and
mp_holistic = mp.solutions.holistic

# Initializing the Model
holistic_model = mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
 
# Initializing the drawing utils for drawing the facial landmarks on image
mp_drawing = mp.solutions.drawing_utils

# Test Camera

In [4]:
# (0) in VideoCapture is used to connect to your computer's default camera
capture = cv2.VideoCapture(0)
 
# Initializing current time and precious time for calculating the FPS
previousTime = 0
currentTime = 0

while capture.isOpened():
    # capture frame by frame
    ret, frame = capture.read()
    
    # resizing the frame for better view
    frame = cv2.resize(frame, (880,720))
 
    # Converting the from BGR to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
 
    # Making predictions using holistic model
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = holistic_model.process(image)
    image.flags.writeable = True
 
    # Converting back the RGB image to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
 
    # Drawing Right hand Land Marks
    mp_drawing.draw_landmarks(
        image,
        results.right_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS
    )
    
    mp_drawing.draw_landmarks(
        image, 
        results.pose_landmarks, 
        mp_holistic.POSE_CONNECTIONS,            
    )
    
    # Drawing Left hand Land Marks
    mp_drawing.draw_landmarks(
        image,
        results.left_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS
    )
    
    # Calculating the FPS
    currentTime = time.time()
    fps = 1 / (currentTime-previousTime)
    previousTime = currentTime
    
    
    # Displaying FPS on the image
    cv2.putText(image, str(int(fps))+" FPS", (10, 70), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
 
    # Display the resulting image
    cv2.imshow("Test Camera", image)

    # Enter key 'q' to break the loop
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break
 
# When all the process is done
# Release the capture and destroy all windows
capture.release()
cv2.destroyAllWindows()

# Create datasheet

In [5]:
model_name = "handModel"
class_name = "rightHand"

#create folder of the model
modelFolerPath = prj_path+"\Dataset/"+model_name
if not os.path.exists(modelFolerPath):
    os.makedirs(modelFolerPath)

In [6]:
# Absolute path of a file
go_src_path = prj_path+"\Dataset\predataset.csv"
go_dst_path = prj_path+"\Dataset\HandModel\predataset.csv"
data_path = prj_path+"\Dataset\HandModel/"+class_name+".csv"

# Copying and rename the file
if not os.path.isfile(data_path):
    shutil.copy(src_path, dst_path)
    os.rename(go_dst_path, data_path)

# Data acquisition

In [59]:
# (0) in VideoCapture is used to connect to your computer's default camera
capture = cv2.VideoCapture(0)
 
while capture.isOpened():
    # capture frame by frame
    ret, frame = capture.read()
 
    # resizing the frame for better view
    frame = cv2.resize(frame, (880,720))
 
    # Converting the from BGR to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
 
    # Making predictions using holistic model
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = holistic_model.process(image)
    image.flags.writeable = True
 
    # Converting back the RGB image to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
 
    # Drawing Right hand Land Marks
    mp_drawing.draw_landmarks(
        image,
        results.right_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS
    )
    
    mp_drawing.draw_landmarks(
        image, 
        results.pose_landmarks, 
        mp_holistic.POSE_CONNECTIONS,            
    )
    
    # Drawing Left hand Land Marks
    mp_drawing.draw_landmarks(
        image,
        results.left_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS
    )
    
    try:
        pose = results.pose_landmarks.landmark
        pose_row = list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in pose]).flatten())

        pose_row.insert(0, class_name)
            
        with open(data_path, mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',')
            csv_writer.writerow(pose_row)
            
    except:
        cv2.putText(image, str('Error')
                    , (210,150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    

    # Display the resulting image
    cv2.imshow("Test Data", image)
 
    # Enter key 'q' to break the loop
    if cv2.waitKey(5) & 0xFF == ord('q') :
        break
 
# When all the process is done
# Release the capture and destroy all windows
capture.release()
cv2.destroyAllWindows()

# TRAIN MODEL

In [7]:
# iterate over files in data directory
list_class = []
list_data = []
directory = modelFolerPath+"/"
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        list_data.append(pd.read_csv(f))
        x = filename.split(".")
        list_class.append(x[0])

In [8]:
sys.stdout.write("[  There is " + str(len(list_class)) + " class for this model  ]\n")
print("==============================================================================")
for i in list_class:
    print(i)

print("==============================================================================")
df_model = pd.concat(list_data)
print("df_model is now complete")

[  There is 3 class for this model  ]
leftHand
normal
rightHand
df_model is now complete


In [9]:
df_model 

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,leftHand,0.492005,0.636674,-0.932085,0.999883,0.518529,0.582139,-0.889761,0.999722,0.536646,...,0.766017,0.000012,0.556676,2.775056,0.297252,0.000051,0.383087,2.756638,0.161466,0.000035
1,leftHand,0.487606,0.639383,-0.850343,0.999890,0.516336,0.584791,-0.814450,0.999738,0.534538,...,0.753507,0.000012,0.556686,2.765715,0.260964,0.000049,0.383100,2.746047,0.144211,0.000034
2,leftHand,0.486890,0.642517,-0.875651,0.999895,0.515651,0.587226,-0.836174,0.999750,0.533838,...,0.686886,0.000013,0.559036,2.763062,0.216515,0.000055,0.384997,2.745802,0.081786,0.000038
3,leftHand,0.484886,0.643399,-0.868381,0.999899,0.514082,0.588030,-0.828943,0.999756,0.532231,...,0.627741,0.000014,0.564689,2.762375,0.178613,0.000059,0.390248,2.745672,0.039937,0.000040
4,leftHand,0.481473,0.643893,-0.885169,0.999901,0.511558,0.588558,-0.847582,0.999761,0.529947,...,0.699985,0.000015,0.563640,2.762819,0.190000,0.000061,0.390280,2.746041,0.104915,0.000042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,rightHand,0.501586,0.645404,-0.886876,0.999943,0.526033,0.588651,-0.826180,0.999918,0.543458,...,0.505441,0.000040,0.609954,2.890220,-0.603158,0.000034,0.421394,2.896685,-0.154797,0.000078
194,rightHand,0.501839,0.645708,-0.899606,0.999945,0.526225,0.588850,-0.836376,0.999920,0.543655,...,0.451714,0.000039,0.629526,2.890566,-0.590895,0.000034,0.441223,2.896758,-0.220161,0.000078
195,rightHand,0.501645,0.645609,-0.899904,0.999945,0.526206,0.588754,-0.837304,0.999920,0.543650,...,0.335054,0.000040,0.625895,2.890768,-0.581632,0.000034,0.440076,2.896679,-0.339751,0.000078
196,rightHand,0.501842,0.645391,-0.923260,0.999946,0.526532,0.588094,-0.857197,0.999922,0.543907,...,0.570394,0.000040,0.621123,2.890824,-0.571184,0.000034,0.429682,2.897241,-0.113302,0.000080


In [10]:
x = df_model .drop('class', axis=1) 
y = df_model ['class']

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1234)

In [53]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression(), verbose = 1),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier(), verbose = 1),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier(), verbose = 1),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier(), verbose = 1),
    'knn':make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=1), verbose = 1),
    'mlp':make_pipeline(StandardScaler(), MLPClassifier(random_state=1, max_iter=300), verbose = 1),
    'gpc':make_pipeline(StandardScaler(),GaussianProcessClassifier(kernel=1.0 * RBF(1.0),random_state=0), verbose = 1),
    'nb':make_pipeline(StandardScaler(),GaussianNB(), verbose = 1),
    'svc':make_pipeline(StandardScaler(),SVC(gamma='auto'), verbose = 1),
    'ac':make_pipeline(StandardScaler(),AdaBoostClassifier(n_estimators=100, random_state=0), verbose = 1),
}

In [54]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(x_train, y_train)
    fit_models[algo] = model

[Pipeline] .... (step 1 of 2) Processing standardscaler, total=   0.0s
[Pipeline]  (step 2 of 2) Processing logisticregression, total=   0.0s
[Pipeline] .... (step 1 of 2) Processing standardscaler, total=   0.0s
[Pipeline] ... (step 2 of 2) Processing ridgeclassifier, total=   0.0s
[Pipeline] .... (step 1 of 2) Processing standardscaler, total=   0.0s
[Pipeline]  (step 2 of 2) Processing randomforestclassifier, total=   0.2s
[Pipeline] .... (step 1 of 2) Processing standardscaler, total=   0.0s
[Pipeline]  (step 2 of 2) Processing gradientboostingclassifier, total=   3.5s
[Pipeline] .... (step 1 of 2) Processing standardscaler, total=   0.0s
[Pipeline]  (step 2 of 2) Processing kneighborsclassifier, total=   0.0s
[Pipeline] .... (step 1 of 2) Processing standardscaler, total=   0.0s
[Pipeline] ..... (step 2 of 2) Processing mlpclassifier, total=   0.2s
[Pipeline] .... (step 1 of 2) Processing standardscaler, total=   0.0s
[Pipeline]  (step 2 of 2) Processing gaussianprocessclassifier,

In [55]:
fit_models['knn'].predict(x_test)

In [56]:
for algo, model in fit_models.items():
    yhat = model.predict(x_test)
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 1.0
rf 1.0
gb 0.9933333333333333
knn 1.0
mlp 1.0
gpc 1.0
nb 0.9666666666666667
svc 1.0
ac 1.0


In [153]:
model_dst=prj_path+"\Model/"
save_dst=model_dst+'test_model.pkl'
pickle.dump(fit_models['rf'],open(save_dst, 'wb'))